# Assignment 2

#### Student ID: *Double click here to fill the Student ID*

#### Name: *Double click here to fill the name*

Firstly, install the following dependencies:

In [ ]:
!pip install shap -qq
!pip install lime -qq
!pip install imodels -qq
!pip install pyngrok -qq
!pip install cleanlab -q
!pip install dtreeviz -qq

If you are using Colab or Kaggle notebook, try to install tensorflow-model-server and set up the tunnel using the following commands:

In [ ]:
import sys
from pyngrok import ngrok, conf
import getpass

if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

    print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
    conf.get_default().auth_token = getpass.getpass()

    # Setup a tunnel to the streamlit port 8050
    public_url = ngrok.connect(8050)

    print(public_url)

For this assignment, besides the labs, you may find the assignments in the previous offering useful https://phonchi.github.io/nsysu-math604/.

In [ ]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer

## Q1: Data cleaning with colic dataset

<center><img src="https://cdn.leonardo.ai/users/f26a2ba8-8273-45e9-8db9-958f83058486/generations/97af1bc3-e1b5-4c6a-84a7-3a8e7a37e97f/Default_Horse_medical_records_and_doctor_0.jpg"></center>

In this problem, we will deal with a colic dataset modified from https://www.kaggle.com/datasets/uciml/horse-colic from our customer. It presents the medical attributes of horses afflicted by colic, detailing their survival outcomes and comprises 300 instances and 26 input variables, along with a single output variable. We will predict whether the problem was surgical or not, making it a binary classification problem. Notably, a significant number of missing values appear across various columns, denoted by a question mark character ("?"). 

Firstly, execute the following code snippet for data preparation:

In [ ]:
# Function for comparing different approaches
def scoring_accuracy(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(random_state=2023)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

In [ ]:
data = pd.read_csv('colic.csv', na_values='?')
data.head()

In [ ]:
# To keep things simple, we'll split the columns into numerical can categorical features
mapping_dict = {
    'no': 0,
    'yes': 1
}

data["surgical_lesion"] = data["surgical_lesion"].replace(mapping_dict)
y = data["surgical_lesion"]
data = data.drop("surgical_lesion", axis=1)
num_col = data.select_dtypes(exclude=['object'])
cat_col = data.select_dtypes(exclude=['int64','float64'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(data, y, train_size=0.8, test_size=0.2, random_state=0)
X_train_num = X_train.select_dtypes(exclude=['object'])
X_valid_num = X_valid.select_dtypes(exclude=['object'])
X_train_cat = X_train.select_dtypes(exclude=['int64','float64'])
X_valid_cat = X_valid.select_dtypes(exclude=['int64','float64'])

To ensure reproducibility, please set all the random seeds to 2023:

#### (a) To simplify the problem, we will consider only the numerical columns first. We will compare the model accuracy between the three data-cleaning approaches. (10%)

1. Replace missing values with the mean value along each column using `SimpleImputer()`.
2. Use the KNN imputation with 3 neighbors using `KNNImputer()`.
3. Use the iterative imputation method and set the regressor as Radnomforest using `IterativeImputer()`.

Use the `scoring_accuracy()` function provided above to perform classification and calculate the accuracy for each approach. Finally, make some comments on the results.

Hint: Since we are working with both training and validation sets, you should apply the same transform when you impute the missing value for both sets.

In [ ]:
# Simple Imputation
# coding your answer here.

In [ ]:
# KNN Imputation
# coding your answer here.

In [ ]:
# Iterative Imputation
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (b) Now, we will add categorical variables into consideration. Try to:

* Replace missing values with the most frequent value along each column for categorical variables. 
* Perform label (ordinal) encoding for the categorical variables. 
* When there are unknown categories in the validation set, set it to -1 for the label encoding.
* Use the best approach you found in (a) to impute the numerical column.

Combining the transformation above, which contains a separate numerical and categorical pipeline to the original training and validation data split (`X_train` and `X_valid`) and using the `scoring_accuracy()` function to calculate the accuracy. Make some comments on the results when comparing with (a). (10%)

Hint: Since we are working with both training and validation sets, try to apply the same transform when you impute the missing data or encode the variables. You may find [ColumnTransformer](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html), [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and the [`handle_unknown`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) option in the encoder useful.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (c) Use `CleanLearning` with `RandomForestClassifier` to fit the cleaned training you obtained in (b) (i.e., after imputation and encoding). Then, calculate the accuracy of the validation set. Finally, report the possible label issue in the training set using `cleanlab.dataset.health_summary()`. (10%)

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q2. Feature engineering and selection with Ames housing dataset

In this question, we are going to examine several feature engineering and feature selection methods.

The dataset is from our previous project, which is a modified version of the Ames housing dataset. The original data was compiled by Dean De Cock for use in data science education and published in [De Cock, D. (2011)](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The modified version contains 2930 rows with 79 columns describing every aspect of residential homes in Ames, Iowa. The target of this problem is the `SalePrice`, while all the other columns are treated as features.

Firstly, execute the following code snippet for data preparation:

In [ ]:
# Function for comparing different approaches with cross validation
def score_dataset(X, y, pipeline):
    # See https://stats.stackexchange.com/questions/27750/feature-selection-and-cross-validation
    # , https://stackoverflow.com/questions/56308116/should-feature-selection-be-done-before-train-test-split-or-after
    # and https://stats.stackexchange.com/questions/2306/feature-selection-for-final-model-when-performing-cross-validation-in-machine

    # For simplicity, we perform label encoding for categoricals here, though it may be better to put it in pipline
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        pipeline, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

df = pd.read_csv("ames.csv")
X = df.copy()
y = X.pop('SalePrice')

X.head()

To ensure reproducibility, please set all the random seeds to 2023:

#### (a) The Ames dataset has a lot of features! Fortunately, you can identify the features with the most potential and use the filtering method. To start with, report the mutual information between the target and each feature in descending order using the `mutual_info_regression()` function on the original dataset `X`. Then, build a pipeline that selects the best 10 features and transforms the dataset using `SelectKBest()` function and fits the resulting dataset using Random forest. Calculate the RMSLE on the transformed dataset using the `score_dataset()`. (10%)


Hint: Since we are working with both training and validation sets, you may find [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) useful.

In [ ]:
# coding your answer here.

#### (b) Next, we'll rely on PCA to try to untangle the correlational structure of these features. Firstly, apply PCA on the dataset `X` and plot the cumulative explain variance using `pca()`. Then, build a pipeline that selects the number of components so that the cumulative explained variance is just above 90%  and fits the resulting dataset using Random forest. Calculate the RMSLE on the transformed dataset using the `score_dataset()`. (10%)

Hint: PCA is scale sensitive. Therefore, you must standardize the dataset before performing PCA by putting it into the pipeline. Since we are working with both training and validation sets, you may find [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) useful.

In [ ]:
# coding your answer here.

#### (c) Finally, we will explore the embedding method. Try to build a pipeline that selects features using `SelectFromModel()` with the feature importance provided by `RandomForestRegressor` and fits the resulting dataset using Random forest. Then, calculate the RMSLE on the transformed dataset using the `score_dataset()`. Finally, comment on the results obtained by comparing it with (a) and (b). (10%)

Hint: You do not need to specify `threshold` or `max_features`; just use the default value to determine the number of features automatically. Since we are working with both training and validation sets, you may find [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) useful.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q3: Analyze breast cancer dataset with interpretable methods

Assume that there is a center that consults us to diagnose breast cancer and has digitized the images of the breast from around 570 patients. Features were computed from these digitized images that described the characteristics of cell nuclei in the images. For each cell nucleus, 10 features are used to describe its characteristics. For all the nuclei present in an image of a patient, the mean, standard error, and the largest or worst values are computed for each of these 10 features. Each patient, therefore, has 30 features in total. Given these input features, the goal of the system is to predict whether the cell is benign or malignant and to provide a confidence score for the doctor to help with their diagnosis.



Firstly, execute the following code snippet for data preparation:

In [ ]:
data = load_breast_cancer(as_frame=True)
X = data['data']
y = data['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=24)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=24)

In [ ]:
X_train.head()

To ensure reproducibility, please set all the random seeds to 2023:

#### (a) First, fit an interpretable model and show doctors some evidence the model is doing something in line with their medical intuition. (10%)

* Try to build a `FIGSClassifier()` decision rule model as a baseline and calculate the accuracy of the model on the validation set. Set the parameter [`max_rule`](https://csinva.io/imodels/tree/figs.html) to 5.
* Draw the feature importance plot using the [feature_importances_](https://csinva.io/imodels/tree/figs.html#imodels.tree.figs.FIGS.feature_importances_) and find out the most important feature.
* Now, a patient (The data is recorded in the first row in the test dataset with id `331`) comes in, and he/she would like to know why he/she got (or does not get) breast cancer. Use the rule generated from the model to give him/her some reasons. You may also use a tree to visualize the decision path.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (b) The doctor is glad you convinced the patients and would like to know more. It appears `worst area` of the cell is a critical feature, and the doctors would like to know more about that. (10%)

* Divide the original data `X` into two datasets, Benign (Target equals 1) and Malignant (Target equals 0), respectively. Draw the distribution of the `worst area` for these two datasets.
* Create a partial dependence plot for them that shows how `worst area` of the cell affects the model's predictions. (You should use the validation set to generate the plot)
* Comment on your results based on the previous two items about how `worst area` of the cell affect the predictions. Is it consistent with the results from (a)?

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (c) The doctor is glad about the results, but he/she is still worried about the model performance of the model you just built.  (5%)

* Try to build a more complicated classifier based on a deep neural network (DNN) and train it using the following code. 
* Then report the accuracy on the validation set and test set.
* Find out the patients that the model is most confident and least confident based on the probability from `model.predict()`, and report their index.

In [ ]:
tf.keras.utils.set_random_seed(2023)
# Create a sequential model
model = models.Sequential()

# Add layers to the model
model.add(layers.Dense(20, input_dim=30, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model for 300 epochs and track the history
num_epochs = 300
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (d) Use Lime and Shap to explain the two patients you find in (c)
* Create the [`lime_tabular.LimeTabularExplainer()`](https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_tabular) and use [`explain_instance()`](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_tabular.LimeTabularExplainer.explain_instance) to explain the two instance. Then, draw the local explanation for these two instances by showing the most important five features.
* Create the [`KernelExplainer()`](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.KernelExplainer.html#) using `shap` with the following argument, where `prob` is also defined below:  (15%)

```python
KernelExplainer(prob, X_train.to_numpy(), link="logit")

def prob(data):
    return model.predict(data).reshape(-1, 1)
```
* Using the above [`KernelExplainer()`](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.KernelExplainer.html#) to explain the two instances by calculating the shap value and draw the force plots for them using [`shap.force_plot()`](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.force_plot.html). You can set the parameter [`nsamples`](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.KernelExplainer.html#shap.KernelExplainer.shap_values) to 100 to speed up the calculation.
* Based on the results of `lime` and `shap` explain why these two patients got (or does not get) breast cancer. Do the results consistent with (a) and (b)?

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

#### (e) Now, the doctors are convinced you have the right data, and the model overview looks reasonable. It's time to turn this into a finished product they can use. (10%)
* Try to use `tf.keras.models.save_model()` to save your DNN model.
* Deploy your model as a REST API server using TensorFlow Serving with `tensorflow_model_server`. 
* Test your server by sending a request that contains the data of the previous two patients using `json.dumps()` and `requests.post()`. Show that the responses from the server are close to the predictions of the original model for these two patients using [`np.isclose()`](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html).

Hint: Notice that the input to the DNN model should be with shape `(number of samples, number of features)` even if you feed a single instance.

In [ ]:
# coding your answer here.